# Using FlexBe with ROS

<img src="img/flexbe_img.png" width="1000" />

<img src="img/robotignite_logo_text.png" width="400"/>

## Unit 2: Action Client and Autonomy Levels

<p style="background:green;color:white;">SUMMARY</p>

Estimated time to completion: <b>3 hours</b><br><br>
In this unit, you are going to learn how to create a new state based on an Action Client, and also how to use the Autonomy Levels in FlexBe.

<p style="background:green;color:white;">END OF SUMMARY</p>

So, in the previous unit, you learned how to create a new FlexBe state, and also how to add a new state to your behavior. At this point, what you have is a FlexBe behavior that allows a mobile robot (the Turtlebot, in this case) to travel a certain distance, taking into account the obstacles that may appear in its path.

But, of course, you can add as many more states as you wish in order to generate a more complex behavior. And that's exactly what we are going to do next. In the state you saw in the previous unit, you used two kinds of FlexBe proxies: the **ProxyPublisher** and the **ProxySubscriberCached**. In the new state we are going to add to our behavior, though, we are going to be using another kind of FlexBe proxy: the **ProxyActionClient**. These state are also known as **Actionlib states**, and are very common in FlexBe development.

The **ProxyActionClient** provides an interface where robot capabilities can be provided by ROS nodes implementing an action server, and states can then access them by acting as an action client. The interface is designed for long-term (longer than one update cycle) actions, as it is non-blocking and optionally provides feedback while being executed. This makes it a perfect interface to be used along with FlexBe, and it is recommended to base your states on action interfaces whenever they do something more complex than just subscribing to a topic.

## Creating an Actionlib state

<p style="background:#EE9023;color:white;">**Exercise 2.1**</p>
<br>

a) First of all, let's add our new state. For that, let's create a new script inside **"turtlebot_behaviors/turtlebot_flexbe_states/src/turtlebot_flexbe_states"**, which we can call **turn_state.py**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
cd src/turtlebot_behaviors/turtlebot_flexbe_states/src/turtlebot_flexbe_states/

In [ ]:
touch turn_state.py;chmod +x turn_state.py

Inside this new script, copy the following content.

<p style="background:green;color:white;">**turn_state.py**</p>

In [ ]:
#!/usr/bin/env python
import rospy
import math
from flexbe_core import EventState, Logger
from flexbe_core.proxy import ProxyActionClient

from spin_action_server.msg import SpinAction, SpinGoal, SpinResult, SpinFeedback

class TurnState(EventState):
    '''
	AvoidObstacleState implements a couple of basic obstacle avoidance algorithms. This state allows the robot to choose a different
    to go to.

    -- turn_angle   float   The angle that the robot should make
	-- t_speed 	float 	Speed at which to turn the robot

	<= done 		    If turn completed successfully

	'''
    def __init__(self, turn_angle, t_speed):
        super(TurnState, self).__init__(outcomes=['done', 'failed'])
        self._turn_angle = turn_angle
        self._t_speed = t_speed
        
        self._topic = '/spin_server_X'
        
        #create the action client passing it the spin_topic_name and msg_type
        self._client = ProxyActionClient({self._topic: SpinAction}) # pass required clients as dict (topic: type)
        
        # It may happen that the action client fails to send the action goal.
        self._error = False
    
    def execute(self, userdata):
        # While this state is active, check if the action has been finished and evaluate the result.
        
        # Check if the client failed to send the goal.
        if self._error:
            return 'failed'
            
        # Check if the action has been finished
        if self._client.has_result(self._topic):
            result = self._client.get_result(self._topic)
            result_val = result.done
            
            # Based on the result, decide which outcome to trigger.
            if result_val == "done":
                return 'done'
            else:
                return 'failed'
        
        # Check if there is any feedback
        if self._client.has_feedback(self._topic):
            feedback = self._client.get_feedback(self._topic)
            Logger.loginfo("Current heading is: %s" % feedback.heading)
            
        # If the action has not yet finished, no outcome will be returned and the state stays active.


    def on_enter(self, userdata):
        # When entering this state, we send the action goal once to let the robot start its work.
        # Create the goal.
        goal = SpinGoal()
        goal.angle = self._turn_angle
        goal.turn_speed = self._t_speed
        
        # Send the goal.
        self._error = False # make sure to reset the error state since a previous state execution might have failed
        try:
            self._client.send_goal(self._topic, goal)
        except Exception as e:
            # Since a state failure not necessarily causes a behavior failure, it is recommended to only print warnings, not errors.
            # Using a linebreak before appending the error log enables the operator to collapse details in the GUI.
            Logger.logwarn('Failed to send the Spin command:\n%s' % str(e))
            self._error = True
        
    def on_exit(self, userdata):
        # Make sure that the action is not running when leaving this state.
        # A situation where the action would still be active is for example when the operator manually triggers an outcome.
        
        if not self._client.has_result(self._topic):
            self._client.cancel(self._topic)
            Logger.loginfo('Cancelled active spin action goal.')
        
    def on_start(self):
        Logger.loginfo("Robot Turn state: READY!")
        
    def on_stop(self):
        Logger.loginfo("Robot Turn: Disengaged!")
        
        #same implementation as on exit
        if not self._client.has_result(self._topic):
            self._client.cancel(self._topic)
            Logger.loginfo('Cancelled active spin action goal.')

<p style="background:green;color:white;">**turn_state.py**</p>

You can have a look at the code above to try to see what it will do. But don't worry if you don't understand the whole code right now. We'll review and explain everything in detail in this chapter.

<p style="background:#EE9023;color:white;">**End of Exercise 2.1**</p>

Let's now have a closer look at the state that we created in the previous exercise, 2.1.

In [ ]:
#!/usr/bin/env python
import rospy
import math
from flexbe_core import EventState, Logger
from flexbe_core.proxy import ProxyActionClient

from spin_action_server.msg import SpinAction, SpinGoal, SpinResult, SpinFeedback

First of all, we can see some imports that we will need for this class. Some of them are similar to the script we saw in the previous unit, like the **EventState** and **Logger** classes from the **flexbe_core** module.

Also, pay attention to the **ProxyActionClient** class, from **flexbe_core.proxy**.

In [1]:
def __init__(self, turn_angle, t_speed):
    super(TurnState, self).__init__(outcomes=['done', 'failed'])
    self._turn_angle = turn_angle
    self._t_speed = t_speed

    self._topic = '/spin_server_X'

    #create the action client passing it the spin_topic_name and msg_type
    self._client = ProxyActionClient({self._topic: SpinAction}) # pass required clients as dict (topic: type)

    #It may happen that the action client fails to send the action goal.
    self._error = False

Next, we have the constructor of the class. As you can see, it takes two parameters as input: **turn_angle** and **t_speed**. These are the angle at which we want our robot to turn and the maximum speed at which we want it done.

Next, we are initializing some variables we'll use on the class, including the inputs we've just talked about, which we also store in variables of the class.

Then, we are storing the topic we are going to use in a variable. In this case, it's the topic of the Action Server we will connect to, which is **/spin_server_X**.

Finally, we define the proxy to connect to the Action Server. So, we are using the **ProxyActionClient**. As you can see, this proxy is also instantiated by passing a dictionary with the topic name to it, and the message to use (**SpinAction**, in this case).

In [ ]:
self._client = ProxyActionClient({self._topic: SpinAction})

Finally, we are initializing the error variable to False.

In [ ]:
def execute(self, userdata):
    # While this state is active, check if the action has been finished and evaluate the result.

    # Check if the client failed to send the goal.
    if self._error:
        return 'failed'

    # Check if the action has been finished
    if self._client.has_result(self._topic):
        result = self._client.get_result(self._topic)
        result_val = result.done

        # Based on the result, decide which outcome to trigger.
        if result_val == "done":
            return 'done'
        else:
            return 'failed'

    # Check if there is any feedback
    if self._client.has_feedback(self._topic):
        feedback = self._client.get_feedback(self._topic)
        Logger.loginfo("Current heading is: %s" % feedback.heading)

    # If the action has not yet finished, no outcome will be returned and the state stays active.

Next we have the **execute()** function. Inside this function, we are checking several things. The workflow goes as follows:

1. First, we check if the value of the variable **self._error** is True. If it is, then we exit the function and return a **failed** state.
2. Second, we check if the Action Server has returned a result yet (has finished). If it has returned a result, then we check the value of that result. If the value is **done**, then we exit and set the state to **done**. If it's not done, then we exit and set the state to **failed**.
3. Finally, we check if there is any feedback from the Action Server. If there is, then we publish that feedback.

In [2]:
def on_enter(self, userdata):
    # When entering this state, we send the action goal once to let the robot start its work.
    # Create the goal.
    goal = SpinGoal()
    goal.angle = self._turn_angle
    goal.turn_speed = self._t_speed

    # Send the goal.
    self._error = False # make sure to reset the error state since a previous state execution might have failed
    try:
        self._client.send_goal(self._topic, goal)
    except Exception as e:
        # Since a state failure does not necessarily cause a behavior failure, it is recommended to only print warnings, not errors.
        # Using a linebreak before appending the error log enables the operator to collapse details in the GUI.
        Logger.logwarn('Failed to send the Spin command:\n%s' % str(e))
        self._error = True

Next we have the **on_enter()** state. This event is called when this state is transitioned to. Here, we are creating the message (goal) to be sent to the Action Server. But note that we are also sending the goal here:

In [ ]:
self._client.send_goal(self._topic, goal)

When working with Actionlib states, the **on_enter()** event is where you should create and also send the goal to the Action Server.

In [3]:
def on_exit(self, userdata):
    # Make sure that the action is not running when leaving this state.
    # A situation where the action would still be active is, for example, when the operator manually triggers an outcome.

    if not self._client.has_result(self._topic):
        self._client.cancel(self._topic)
        Logger.loginfo('Cancelled active spin action goal.')

The **on_exit()** event is called when leaving this state. Here we are cancelling the goal, if it is still active. 

In [ ]:
def on_start(self):
    Logger.loginfo("Robot Turn state: READY!")

def on_stop(self):
    Logger.loginfo("Robot Turn: Disengaged!")

    #same implementation as on exit
    if not self._client.has_result(self._topic):
        self._client.cancel(self._topic)
        Logger.loginfo('Cancelled active spin action goal.')

In the **on_start()** and **on_stop()** event implementations of this state, we are just printing out some log messages and cancelling the goal (again), if it is still active.

And that's it! So, at this point, you have created your first FlexBe Actionlib state. But before being able to use this state, you'll need something else, don't you think? Do you know what's missing here?

Yes! That's right! The Action Server is missing! So, the next step will be to create our Action Server.

<p style="background:#EE9023;color:white;">**Exercise 2.2**</p>
<br>

a) First of all, let's create a new package where we will put all the files and code required by our Action Server.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd; cd ../src

In [ ]:
catkin_create_pkg spin_action_server rospy actionlib actionlib_msgs

b) Inside the **src** folder of the new package, create a new file named **spin_server.py**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd; cd ../src/spin_action_server/src

In [ ]:
touch spin_server.py

Inside this new file, copy the following content:

<p style="background:green;color:white;">**spin_server.py**</p>

In [ ]:
#! /usr/bin/env python

import rospy
import actionlib
from spin_action_server.msg import SpinAction, SpinGoal, SpinResult, SpinFeedback
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist
from tf import transformations
import math

class TurnRobotAction(object):
    def __init__(self):
        self._as = actionlib.SimpleActionServer("spin_server_X", SpinAction, execute_cb=self.execute_cb, auto_start = False)
        
        self.odom_data = None
        self.initial_orientation = None
        self.cur_orientation = None
        self._turn = False
        self._turn_angle = None
        self.cmd_pub = Twist()
        self._feedback = SpinFeedback()
        self._result = SpinResult()
        
        self.sub = rospy.Subscriber("/odom", Odometry, self.odom_callback)
        self.pub = rospy.Publisher("/cmd_vel", Twist, queue_size=10 )
        self._as.start()
    
    def execute_cb(self, goal):
        r = rospy.Rate(30)
        self._turn_angle = math.fabs((goal.angle * math.pi)/ 180)
        rospy.loginfo("Target Robot Heading: %s" % self._turn_angle)
        
        #set initial orientation
        if not self.initial_orientation:
            self.initial_orientation = self.cur_orientation
            self._turn = True
        
        while (self.initial_orientation and self._turn):
            #skip if initial and current orientation is same
            if self.initial_orientation == self.cur_orientation:
                continue
            else:
                #calculate the differences in orientation
                cur_angle = transformations.euler_from_quaternion((self.cur_orientation.x, self.cur_orientation.y, 
                self.cur_orientation.z, self.cur_orientation.w))
        
                start_angle = transformations.euler_from_quaternion((self.initial_orientation.x, self.initial_orientation.y, 
                self.initial_orientation.z, self.initial_orientation.w))
                
                turn_so_far = math.fabs(cur_angle[2] - start_angle[2])
                #rospy.loginfo("Current Robot turn at: %f" % turn_so_far)
                #send feedback (current Robot Heading)
                self._feedback.heading = turn_so_far
                self._as.publish_feedback(self._feedback)
                rospy.loginfo("Current Robot Heading: %s" % turn_so_far)
                
                if (turn_so_far >= self._turn_angle):
                    #reset some variables if done turning
                    self._turn = False
                    self.initial_orientation = None
                    self._result.done = "done"
                    self._as.set_succeeded(self._result)
                    rospy.loginfo("Turn successfully completed!")
        
            #turn
            if self._turn:
                self.cmd_pub.linear.x = 0.0
                if goal.angle > 0:
                    self.cmd_pub.angular.z = goal.turn_speed
                else:
                    self.cmd_pub.angular.z = -goal.turn_speed
                self.pub.publish(self.cmd_pub)
            
            else: #stop the robot before breaking out
                self.cmd_pub.angular.z = 0.0
                self.pub.publish(self.cmd_pub)
                break
            
            r.sleep()
    
    def odom_callback(self, data): #get the odometry data
        #get current orientation
        self.cur_orientation = data.pose.pose.orientation
        


if __name__ == '__main__':
    rospy.init_node('spin_node')
    rospy.loginfo("Spin Server started ...")
    server = TurnRobotAction()
    rospy.spin()

<p style="background:green;color:white;">**spin_server.py**</p>

You can have a look at the code above and try to understand how it works. But basically, all you need to know is that it allows a robot to turn at a certain angle (specified by the user) at a certain speed (also specified by the user )on its z-axis. 

c) With the code for our Action Server created, let's now generate the messages it will use. First of all, let's create an **action** directory, where we will place the file that will define our message.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd; cd ../src/spin_action_server

In [ ]:
mkdir action

Now, create a file named **Spin.action** inside the **action** folder.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd; cd ../src/spin_action_server/action

In [ ]:
touch Spin.action

Copy the below contents into it.

<p style="background:green;color:white;">**Spin.action**</p>

In [ ]:
float64 angle
float64 turn_speed
---
string done
---
float64 heading

<p style="background:green;color:white;">**Spin.action**</p>

d) Now, before being able to compile the package, you'll need to do some modifications to the **CmakeLists.txt** and **package.xml** files.

### <span style="color: red;">If you don't know exactly how to proceed here, you can have a look at Unit 4 (Part 2) of the ROS Basics in 5 Days Course. There you will find detailed instructions on how to modify these two files. </span>

Here you have minimum examples of how your files should look:

<p style="background:green;color:white;">**CMakeLists.txt**</p>

In [ ]:
cmake_minimum_required(VERSION 2.8.3)
project(spin_action_server)

## Find catkin macros and libraries
## if COMPONENTS list like find_package(catkin REQUIRED COMPONENTS xyz)
## is used, also find other catkin packages
find_package(catkin REQUIRED COMPONENTS
  actionlib
  actionlib_msgs
  message_generation
  message_runtime
  rospy
)

## Generate actions in the 'action' folder
add_action_files(
   FILES
   Spin.action
#   Action2.action
)

## Generate added messages and services with any dependencies listed here
 generate_messages(
   DEPENDENCIES
   actionlib_msgs
 )

catkin_package(
#  INCLUDE_DIRS include
  LIBRARIES spin_action_server
  CATKIN_DEPENDS actionlib actionlib_msgs message_generation message_runtime rospy
  DEPENDS system_lib
)

## Specify additional locations of header files
## Your package locations should be listed before other locations
# include_directories(include)
include_directories(
  ${catkin_INCLUDE_DIRS}
)

<p style="background:green;color:white;">**CMakeLists.txt**</p>

<p style="background:green;color:white;">**package.xml**</p>

In [ ]:
<?xml version="1.0"?>
<package>
  <name>spin_action_server</name>
  <version>0.0.0</version>
  <description>The spin_action_server package</description>
  <maintainer email="user@todo.todo">user</maintainer>
  <license>TODO</license>

  <buildtool_depend>catkin</buildtool_depend>
  <build_depend>actionlib</build_depend>
  <build_depend>actionlib_msgs</build_depend>
  <build_depend>message_generation</build_depend>
  <build_depend>message_runtime</build_depend>
  <build_depend>rospy</build_depend>
  <run_depend>actionlib</run_depend>
  <run_depend>actionlib_msgs</run_depend>
  <run_depend>message_runtime</run_depend>
  <run_depend>message_generation</run_depend>
  <run_depend>rospy</run_depend>

  <export>
  </export>

</package>


<p style="background:green;color:white;">**package.xml**</p>

e) Finally, let's compile the new package so that all the messages are generated.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd; cd ..

In [ ]:
catkin_make

In [ ]:
source devel/setup.bash

<p style="background:#EE9023;color:white;">**End of Exercise 2.2**</p>

Great! So... at this point, we have almost created the whole structure needed to execute our Actionlib state. This includes the **Action Server** that will provide the functionality to turn our robot and the **Actionlib State** that will connect to the Action Server in order to use that functionality. There's only one thing remainining...

We need to add our new state to FlexBe! And that's exactly what we are going to do next.

## Adding the Actionlib state to FlexBe

Let's try to use our new state in FlexBe! For that, you'll need to launch the FlexBe application.

<p style="background:#EE9023;color:white;">**Exercise 2.3**</p>

a) Launch the FlexBe application and add your new state, as shown in the previous chapter. You should have something like this:

<img src="img/flexbe_turn1.png" width="800" />

b) Now, reconnect the **failed** output to your new state.

<img src="img/flexbe_turn2.png" width="800" />

Also, connect the **done** output of your new state back to the Drive State. You should end up with something like this:

<img src="img/flexbe_turn3.png" width="800" />

c) Finally, connect the **failed** output of your state to the **failed** end point.

<img src="img/flexbe_turn4.png" width="800" />

d) Now, let's generate your parameters. Remember that for this new state, we had two parameters: the turning angle and the turning speed. 

In [ ]:
def __init__(self, turn_angle, t_speed):

Follow the same process as in the previous chapter. At the end, you should have something similar to this:

<img src="img/flexbe_turn_params.png" width="400" />

**IMPORTANT**: Remember to first click on the **Apply** button in order to apply your latest modifications to the behavior, and also click on the **Save Behavior** button to save your behavior.

g) And finally, the last step will be to actually run our behavior. So, let's move to the **Runtime Control** window. You should see something like this:

<img src="img/flexbe_turn_runtime.png" width="800" />

Here, you will be able to execute your behavior by clicking on the **Start Execution** button. Also, as you can see, you can modify the values of the parameters as you like. So, let's set the turning angle to **90 degrees** and the turning speed to 0.4. Then, let's click on the **Start Execution** button and see our new behavior in action!

You should see the Turtlebot robot moving forwards and, once it's too close to the wall, stopping and doing a turn of 90 degrees. Then, it will continue moving forward the specified distance.

<img src="img/flexbe3.gif" width="600" />

**IMPORTANT**: Remember to first start the Action Server before executing your state. If you don't start the Action Server, your state will **NOT WORK**.

<span style="color:red;">**¡¡VERY IMPORTANT!!** Remember that, whenever you are done with the Exercise, you can reset the robot's position by hitting on the button **Reset the simulation!**, at the top-right corner of the simulation screen.</span>

<img src="img/reset_sim.png" width="300" />

<p style="background:#EE9023;color:white;">**End of Exercise 2.3**</p>

## Autonomy Levels

Just as the name implies, the Autonomy Level allows the operator to have complete control over the robot's behavior. Giving the robot some level of autonomy allows it to make its own decisions without operator involvement.

Let's now try to rerun the behavior from the previous exercise, but applying some Autonomy Levels to it.

<p style="background:#EE9023;color:white;">**Exercise 2.4**</p>

a) Launch the FlexBe application again and move to the **Statemachine Editor** panel. Once there, select your new Turning state and a panel will appear on the right side.

<img src="img/flexbe_autonomy1.png" width="800" />

b) Now, modify the Autonomy Level of the **done** transition and set it to **High**. Remember to click the **Apply** button when finished.

<img src="img/flexbe_autonomy2.png" width="400" />

c) Let's now go back to the **Runtime Control** panel. Execute the behavior again and see what happens.

<img src="img/flexbe2.gif" width="600" />

Surprised? The robot has stopped right after doing the 90-degree turn. But why?

The **Autonomy Level was the reason** why the behavior execution stopped during the first run. We just allowed transitions requiring less than low autonomy, which only applies to the most basic transitions. Each transition that required at least low autonomy was blocked and our second transition requires high autonomy as we configured in the last tutorial. This means that, as observed, the behavior will not execute the transition autonomously, but instead indicates that it is ready to be executed and waits for permission.

d) Now that we know a little bit more about what happened in the previous step, let's try to execute the behavior again. Now, when the robot stops after the turn, have a look at the **Runtime Control** window. You will see that there is a transition highlighted.

<img src="img/flexbe_autonomy3.png" width="400" />

Allow the requested outcome by clicking on the highlighted arrow. You can click on outcomes at any time, even if they are not highlighted, in order to force them. The behavior will immediately return outcomes requested in this way, so it is your responsibility to decide whether an outcome should be forced. After permitting the outcome, the behavior will continue as usual. Interesting, right?

d) Let's have a look again at the **Runtime Control** panel. Here at the top-right corner of the screen, you will see a section like this one:

<img src="img/flexbe_autonomy4.png" width="400" />

Within that section, you can read: **Block transitions which require at least X autonomy**. Set this value to **Full**.

<img src="img/flexbe_autonomy6.png" width="400" />

g) Execute your behavior again and see what happens now.

The behavior executed as usual, without requesting any permission for executing transitions, right? That's because we have increased the block level to **Full** so that it will only block transitions with a Full level of autonomy. Since our transition has an Autonomy Level of **High**, it won't be blocked now and everything will run as usual.

<p style="background:#EE9023;color:white;">**End of Exercise 2.4**</p>

## Congratulations!! You are now capable of creating your own Actionlib state, and also managing the Autonomy Levels of your behaviors!